In [1]:
import glob
import pandas as pd
import numpy as np

import helpers
#help(helpers)

# Load

In [2]:
# Get file list load of all csv[case insensitive] files in mydir.
mydir = "data/"
file_list=glob.glob(mydir + '*.[cC][sS][vV]')

df_combined_from_load = helpers.load_csvs(file_list)

# Make sure the date column is a date.
df_combined_from_load["Date"] = pd.to_datetime(df_combined_from_load["Date"])

# with pd.option_context("display.max_rows", 1200):
#    display(combined)

loading: data/CreditCard_1.csv
    Statement Adapter type: credit_card_one
loading: data/Bank1.csv
    Statement Adapter type: bank_one


# Filter

In [3]:


# add a month and a year column for grouping in pivots
df_combined_from_load["Transaction Month"]=df_combined_from_load["Date"].dt.strftime('%Y-%m')
df_combined_from_load["Transaction Year"]=df_combined_from_load["Date"].dt.strftime('%Y')


df_combined_from_load=df_combined_from_load.sort_values(by ='Date')
max_date_from_data = df_combined_from_load['Date'].max()

print("Data Date Range",df_combined_from_load['Date'].min(),max_date_from_data)

Data Date Range 2018-01-01 00:00:00 2019-12-30 00:00:00


# Pivot table to show spending per category

In [4]:
# Limit data to previous number of months for display purposes. 
months_to_view=6

max_date_monthend = max_date_from_data+pd.offsets.MonthEnd(0)

# End date is the end of the month so add one day to roll to first of the next month
start_date = (max_date_monthend-pd.DateOffset(months=months_to_view))+pd.offsets.MonthEnd(0)+pd.DateOffset(days=1)
print("Pivot date range",start_date,"-",max_date_monthend)
df = helpers.filter_by_date(df_combined_from_load,start_date,max_date_monthend)

df_monthly_view = pd.pivot_table(
    df,
    index=["AutoCategory"],
    values=["Amount"],
    columns=["Transaction Month"],
    dropna=True,
    aggfunc={"Amount":[np.sum]},fill_value=0)

Pivot date range 2019-07-01 00:00:00 - 2019-12-31 00:00:00


# Get this year's and last year's mean per category


In [5]:
# Get the previous year to the most recent year. Spending grouped by AutoCategory 
# and divided by 12 to get monthly mean
max_year = pd.Period(df_combined_from_load["Transaction Year"].max()) 
previous_year = max_year-1
df_filtered = helpers.filter_by_date(df_combined_from_load,previous_year.start_time,previous_year.end_time)


# add a mean column for each of the Auto Categories
df_last_year = df_filtered.groupby([df_filtered["AutoCategory"]]).sum()/12
# rename MultiIndex ['Amount'] column to more decriptive name
df_last_year.columns = pd.MultiIndex.from_product([["last_years_mean"], [''], ['']])


# Get all or part of the most recent year spending grouped by AutoCategory 
# and divided by the most recent month number to get monthly mean
df_filtered = helpers.filter_by_date(df_combined_from_load,max_year.start_time,max_year.end_time)

# add a mean column for each of the Auto Categories
df_this_year=df_filtered.groupby([df_filtered["AutoCategory"]]).sum()/max_date_monthend.month
# rename MultiIndex ['Amount'] column to more decriptive name
df_this_year.columns = pd.MultiIndex.from_product([["this_years_mean"], [''], ['']])

# match on "AutoCategory" and concat in last years mean and this years mean columns
mean_cols = pd.concat([df_this_year, df_last_year], axis=1,sort=False)
mean_cols=mean_cols.fillna(0)

# Display DataFrame with highlights

In [9]:
# turn red if value is greater than last_years_mean.
def highlight_max(s):
    '''
    highlight the maximum in a Series red.
    '''
    is_max = s['last_years_mean']
    #is_max = s == s.max()
    return ['color: red' if v>is_max[0]  else '' for v in s]


# concat mean columns onto the monthy view columns. Adding 0s for "AutoCategory" rows that
# dont have data in the monthy view
df_display = pd.concat([mean_cols, df_monthly_view], axis=1,sort=False)
df_display=df_display.fillna(0)

df_display["diff"] = mean_cols["this_years_mean"]-df_display["last_years_mean"]
# rearrange columns
cols = ['Amount','this_years_mean','last_years_mean','diff']
df_display = df_display[cols]
df_display.sort_values(by=['diff'],  ascending=False , inplace=True)
df_display.loc["Total"] = df_display.sum()
df_display.style.apply(highlight_max, axis=1)


# View Data by AutoCategory

In [7]:
category_to_view = 'None'

start_date = previous_year.start_time
end_date = max_year.end_time

# Apply date and category_to_view filters to data
df_filtered = helpers.filter_by_date(df_combined_from_load,start_date,end_date)
df_filtered=df_filtered[df_filtered["AutoCategory"] == category_to_view] 

# Remove month and year columns from output
df_filtered = df_filtered[['Date','Description','Amount']]

df_filtered=df_filtered.sort_values(by =['Date'], ascending=[False])
with pd.option_context("display.max_rows", 1200):
   display(df_filtered)

,Date,Description,Amount
85,2019-12-30,Check 1180,130.75
10,2019-11-24,Credit Card Fees,25.81
84,2019-11-01,Check 1178,163.42
9,2019-10-10,Credit Card Fees,111.29
83,2019-10-01,Check 1177,78.95
82,2019-08-05,Check 1175,105.18
81,2019-07-01,Check 1174,138.52
8,2019-05-28,Credit Card Fees,69.98
80,2019-05-12,Check 1172,115.39
79,2019-04-18,Check 1171,176.59
